# Chapter 06: Stacks, Queues, and Deques

## 6.1 Stacks

A **stack** is a collection of objects that are inserted and removed according to the ***last-in, first-out (LIFO)*** principle.

#### Example 6.1:

Internet web browsers store the addresses of recently visited sites in stack. Each time a user visits a new site, that site's address is "pushed" onto the stack of addresses. The browser then allows the user to "pop" back to previously visited sites using the "back" button.

#### Example 6.2:
Text editors usually provide an "undo" mechanism that cancels recent editing operations and reverts to former states of a document. This undo operation can be accomplished by keeping text changes in a stack.

### 6.1.1 The Stack Abstract Data Type

Fomally, a stack is an abstrat data type (ADT) such that an instance $\mathcal{S}$ supports the folloiwng two methods:

* `S.push(e)`: Add element `e` to the top of stack `S`.
* `S.pop()`: Remove and return the top element form the stack `S`; an error occurs if the stack is empty.

Additionally, let us define the following accessor methods for convenience:

* `S.top()`: Return a reference to the top element of stack `S`, without removing itl an error occurs if the stack is empty.
* `S.is_empty()`: Return `True` if stack `S` does not contain any elements.
* `len(S)`: Return the number of elements in stack `S`

By convention, we assume that a newly created stack is empty, and that there is no a priori bound on the capacity of the stack. Elements added to the stack can have arbitrary type.

### 6.1.2 Simple Array-Based Stack Implementation

The `list` class already supports adding an element to the end with `append` method, and removing the last element with the `pop` method, so it is natural to align the top of the stack at the end of the list.

Although a aprogrammer could directly use the `list` class in place ofa formal stack class, lists also include behaviors that would break the abstraction that the stack ADT represents. ALso, the terminology used by the `list` class does not precisely align with traditional nomenclature for a stack ADT, in particular the distinction between `append` and `push`. Instead, we demonstrate how to use a list for internal storage while providing a public interface consistent with a stack.

#### The Adapter Pattern

The ***adapter*** design pattern applies to any context where we effectively want to modify an existing class so that its methods match those of a related, but different, class or interface. One general way to apply the adapter pattern is to define a new class in such a way that it contains an instance of the existing class as a hidden field, and then to implement each method of the new class using methods of this hidden instance variable. By applying the adapter pattern in this way, we have created a new class that performs some of the same functions as an existing class, but repackaged in a more convenient way. In the context of tehe stack ADT, we c an adapt Python's list class using the correspondence shown in 

|stack Method|Realization with Python List|
|---|---|
|`S.push(e)`|`L.append(e)`|
|`S.pop()`|`L.pop()`|
|`S.top()`|`L[-1]`|
|`S.is_empty()`|`len(L)==0`|
|`len(S)`|`len(L)`|

In [4]:
class Empty(Exception):
    pass

In [8]:
class ArrayStack:
    """LIFO Stack implementatino using a Python list as underlying storage."""
    
    def __init__(self):
        self._data = []
        
    def __len__(self):
        return len(self._data)
    
    def is_empty(self):
        return len(self._data) == 0
    
    def push(self, e):
        self._data.append(e)
        
    def top(self):
        if self.is_empty():
            raise Empty('Stack is empty')
        return self._data[-1]
    
    def pop(self):
        if self.is_empty():
            raise Empty('Stack is empty')
        return self._data.pop()

#### Analyzing the Array-Based Stack Implementation

|Operation|Running Time|
|---|---|
|`S.push(e)`|$O(1)$ (amozrtized)|
|`S.pop()`|$O(1)$ (amortized)|
|`S.top()`|$O(1)$|
|`S.is_empty()`|$O(1)$|
|`len(S)`|$O(1)$|

### 6.1.3 Reserving Data Using a Stack

As a consequence of the LIFO protocol, a stack can be used as a general tool to reserve a data sequence.

In [9]:
def reverse_file(filename):
    S = ArrayStack()
    original = open(filename)
    for line in original:
        S.push(line.rstrip('\n'))
    original.close()
    
    output = open(filename, 'w')
    while not S.is_empty():
        output.write(S.pop() + '\n')
    output.close()

### 6.1.4 Matching Paratheses and HTML Tags

#### An algorithm for Matching Delimiters

In [35]:
def is_matched(expr):
    
    lefty = "({["
    righty = ")}]"
    
    S = ArrayStack()
    
    for c in expr:
        if c in lefty:
            S.push(c)
        elif c in righty:
            if S.is_empty():
                return False
            if righty.index(c) != lefty.index(S.pop()):
                return False
    return S.is_empty()

In [36]:
is_matched('[(5+x)-(y+z)]')

True

#### Matching Tags in Markup Language

In [37]:
def is_matched_html(raw):
    S = ArrayStack()
    j = raw.find('<')
    
    while j != -1:
        k = raw.find('>', j+1)
        if k == -1:
            return False
        tag = raw[j+1:k]
        if not tag.startswith('/'):
            S.push(tag)
        else:
            if S.is_empty():
                return False
            if tag[1:] != S.pop():
                return False
        j = raw.find('<', k+1)
    return S.is_empty()

## 6.2 Queues

Another fundamental data structure is the ***queue***. It is a close "cousin" of the stack, as a queue is a collection of objects that are inserted and removed according to the ***first-in, first-out (FIFO)*** principle. That is, elements can be inserted at any time, but only the element that has been in the queue the longest can be next removed.

### 6.2.1 The Queue Abstract Data Type

Formally, the queue abstract data type defines a collection that keeps objects in a sequence, where element access and deleteion are restricted to the ***first*** element in the queue, and element insertion is restricted to the back of the sequence. This restriction enforces the rule that items are inserted and deleted in a queue according to the first-in, first-out (FIFO) principle. The ***queue*** abstract data type (ADT) supports the following two fundamental mthods for a queue Q:


* `Q.enqueue(e)`: Add element `e` to the back of queue `Q`
* `Q.dequeue()`: Remove and return the first element form queue `Q`; an error occurs if the queue is empty.
* `Q.first()`: Return a reference to the element at the front of queue `Q`, without removing it; an error occurs if the queue is empty.
* `Q.is_empty()`: Return `True` if queue `Q` does not contain any elements.
* `len(Q)`: Return the number of elements in queue `Q`

By convention, we assume that a newly created queue is empty, and that there is no a priori bound on the capacity of the queue. Elements added to the queue can have arbitrary type.

### 6.2.2 Array-Based Queue Implementation

In [46]:
class ArrayQueue:
    
    DEFAULT_CAPACITY = 10
    
    def __init__(self):
        self._data = [None] * ArrayQueue.DEFAULT_CAPACITY
        self._size = 0
        self._front = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def first(self):
        if self.is_empty():
            raise Empty("Queue is empty")
        return self._data[self._front]
    
    def dequeue(self):
        
        if self.is_empty():
            raise Empty("Queue is empty")
        answer = self._data[self._front]
        self._data[self._front] = None
        self._front = (self._fornt +1) % len(self._data)
        self._size -= 1
        
        if 0 < self._size < len(self._data) // 4:
            self._resize(len(self._data) // 2)
        return answer
    
    def enqueue(self, e):
        if self._size == len(self._data):
            self._resize(2 * len(self._data))
        avail = (self._fornt + self._size) % len(self._data)
        self._data[avail] = e
        self._size += 1
    
    def _resize(self, cap):
        
        old = self._data
        self._data = [None] * cap
        walk = self._front
        for k in range(self._size):
            self._data[k] = old[walk]
            walk = (1 + walk) % len(old)
        self._front = 0

#### Analyzing the Array-Based Queue Implementation

|Operation|Running Time|
|---|---|
|`Q.enqueue(e)`|$O(1)$ (amortized)|
|`Q.dequeue()`|$O(1)$ (amortized)|
|`Q.first()`|$O(1)$|
|`Q.is_empty()`|$O(1)$|
|`len(Q)`|$O(1)$|

## 6.3 Double-Ended Queues

We next consider a queue-like data structure that supports insertion and deletion at both the front and the back of the queue. Such a structure is called a ***double-ended queue***, or ***deque***, which is usally pronounced "deck" to avoid confusion with the `dequeue` method of the regular queue ADT, which is pronounced like the abbreviation "D.Q."

### 6.3.1 The Deque Abstract Data Type

To provide a symmetrical abstraction, the deque ADT is defined so that deque `D` supports the following methods:

* `D.add_first(e)`: Add element `e` to the fornt of deque `D`.
* `D.add_last(e)`: Add element `e` to  the back of deque `D`.
* `D.delete_first()`: Remove and return the fist element from deque `D`
* `D.first()`: Return the first element of deque `D`
* `D.last()`: Return the last element of deque `D`
* `D.is_empty()`: Return `True` if deque `D` does not contain any elements.
* `len(D)`: Return the number of elements in deque `D`

### 6.3.2 Implementing a Deque with a Circular Array

Do it at Exercise P-6.32!

### 6.3.3 Deques in the Python Collections Module

An implementation fo a `deque` class is aviailable in Python's standard collections module. A summary of the most commonly used behaviors of the `collections.deque` class is given follow:

|`collections.deque`|Description|
|---|---|
|`len(D)`|number of elements|
|`D.appendleft()`|add to beginning|
|`D.append()`|add to end|
|`D.popleft()`|remove from beginning|
|`D.pop()`|remove from end|
|`D[0]`|access first element|
|`D[-1]`|access last element|
|`D[j]`|access arbitrary entry by index|
|`D[j] = val`|modify arbitrary entry by index|
|`D.clear()`|clear all contents|
|`D.rotate(k)`|circularly shift rightward $k$ steps|
|`D.remove(e)`|remove first matching element|
|`D.count(e)`|count number of matches for `e`|

The library `deque` constructor also supports an optional `maxlen` parameter to force a fixed-length deque. However, if a call to append at either end is invoked when the deque is full, it does not throw an error; instead, it causes one element to be dropped from the opposite side. That is, calling `appendleft` when the deque is full casuse an implicit `pop` from the right side to make room for the new element.